In [ ]:
!pip3 install flask-ngrok
!pip install dill
import dill
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Загружаем обученные модели
with open("/content/drive/My Drive/logreg_pipeline.dill", "rb") as f:
    pipeline = dill.load(f)

# Загружаем датасет
df = pd.read_csv('/content/drive/My Drive/train.csv')

In [ ]:
from flask_ngrok import run_with_ngrok  # импортируем run_with_ngrok для работы с ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2P9BFWUgU9oxs7G3oilJVYriN93_6VtiGedoctUkoBcuZQvE
!./ngrok http 80

In [ ]:
# Создаем Flask приложение
app = Flask(__name__)
run_with_ngrok(app)  # используем run_with_ngrok для проксирования нашего Flask-приложения через ngrok

@app.route('/', methods=['GET'])
def general():
    return 'Welcome to the prediction process.'

@app.route('/predict', methods=['POST'])
def predict():
    # Получаем данные из JSON запроса
    data = request.json
    input_text = data['text']

    # Применяем обученную модель к входным данным
    predictions = pipeline.predict_proba(pd.DataFrame({'comment_text': [input_text]}))

    # Создаем ответ в формате JSON
    response = {
        'toxic': predictions[0, 0],
        #'severe_toxic': predictions[0, 1],
        #'obscene': predictions[0, 2],
        #'threat': predictions[0, 3],
        #'insult': predictions[0, 4],
        #'identity_hate': predictions[0, 5]
    }

    return jsonify(response)

if __name__ == '__main__':
    app.run()  
